#  Advanced Lens Correction System

Professional solution for Kaggle's Automatic Lens Correction competition. 
Multi-stage pipeline for correcting barrel distortion without lens profiles.
Optimized for geometric accuracy metrics (Edge Similarity, Line Straightness).

**Key Features:**
- Intelligent distortion detection (Edge + Hough + Gradient)
- Multi-stage geometric correction
- Quality enhancement (CLAHE + bilateral filtering)
- Competition-ready output (1000 images in ~12 min)

**Output:** corrected_images.zip + submission.csv

# Cell 1: Notebook Metadata and Setup


In [1]:
# =============================================================================
# 🏆 AUTOMATIC LENS CORRECTION - PROFESSIONAL KAGGLE SOLUTION v3.1
# =============================================================================
# Competition: Kaggle - Automatic Lens Correction
# Goal: Correct barrel distortion in raw images without lens profiles
# Author: Professional Computer Vision Engineer
# Date: February 2026
# Version: 3.1 (BALANCED Mode - Optimized for Competition)
# =============================================================================

print("""
╔════════════════════════════════════════════════════════════════════════════╗
║     ADVANCED LENS CORRECTION SYSTEM - PROFESSIONAL EDITION v3.1           ║
╠════════════════════════════════════════════════════════════════════════════╣
║  • BALANCED distortion detection & correction                             ║
║  • Optimized parameters for competition metrics                           ║
║  • Multi-stage quality enhancement pipeline                               ║
║  • Competition-optimized for geometric accuracy                           ║
║  • Production-ready with comprehensive error handling                     ║
╚════════════════════════════════════════════════════════════════════════════╝
""")


╔════════════════════════════════════════════════════════════════════════════╗
║     ADVANCED LENS CORRECTION SYSTEM - PROFESSIONAL EDITION v3.1           ║
╠════════════════════════════════════════════════════════════════════════════╣
║  • BALANCED distortion detection & correction                             ║
║  • Optimized parameters for competition metrics                           ║
║  • Multi-stage quality enhancement pipeline                               ║
║  • Competition-optimized for geometric accuracy                           ║
║  • Production-ready with comprehensive error handling                     ║
╚════════════════════════════════════════════════════════════════════════════╝



# Cell 2: Import Libraries and Dependencies


In [2]:
# =============================================================================
# 📚 IMPORT LIBRARIES
# =============================================================================

import numpy as np
import pandas as pd
import cv2
import os
import sys
import zipfile
import warnings
import multiprocessing
import psutil
import subprocess
from pathlib import Path
from tqdm import tqdm
from skimage import exposure, filters, morphology, measure, feature
from scipy import ndimage, signal
from datetime import datetime
import matplotlib.pyplot as plt
import skimage

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# Check versions
print(f"✅ OpenCV Version: {cv2.__version__}")
print(f"✅ NumPy Version: {np.__version__}")
print(f"✅ Scikit-image Version: {skimage.__version__}")
print(f"✅ CPU Cores Available: {multiprocessing.cpu_count()}")
print(f"✅ RAM Available: {psutil.virtual_memory().available / (1024**3):.2f} GB")

print("\n✅ All libraries imported successfully!")

✅ OpenCV Version: 4.12.0
✅ NumPy Version: 2.0.2
✅ Scikit-image Version: 0.25.2
✅ CPU Cores Available: 4
✅ RAM Available: 30.10 GB

✅ All libraries imported successfully!


# Cell 3: Path Configuration and Auto-Detection


In [3]:
# =============================================================================
# 🔍 AUTO-DETECT PATHS AND CONFIGURE ENVIRONMENT
# =============================================================================

print("="*60)
print("🔍 CONFIGURING PATHS AND ENVIRONMENT")
print("="*60)

def setup_paths():
    """
    Automatically detect and configure all necessary paths
    """
    base_input = '/kaggle/input'
    
    # Find competition folder
    competition_folders = [f for f in os.listdir(base_input) 
                          if 'automatic-lens-correction' in f.lower()]
    
    if not competition_folders:
        raise Exception("❌ Competition data not found! Please add the competition input.")
    
    COMP_PATH = os.path.join(base_input, competition_folders[0])
    print(f"✅ Competition path: {COMP_PATH}")
    
    # Find train and test folders
    contents = os.listdir(COMP_PATH)
    TRAIN_PATH = None
    TEST_PATH = None
    
    # Look for test folder (prioritize folders containing images)
    for item in contents:
        item_path = os.path.join(COMP_PATH, item)
        if os.path.isdir(item_path):
            files = os.listdir(item_path)
            if files and any(f.endswith(('.jpg', '.png', '.jpeg')) for f in files):
                if 'test' in item.lower() or 'original' in item.lower():
                    TEST_PATH = item_path
                    print(f"✅ Test folder found: {TEST_PATH}")
                elif 'train' in item.lower():
                    TRAIN_PATH = item_path
                    print(f"✅ Train folder found: {TRAIN_PATH}")
    
    # Fallback: use any folder with images as test folder
    if not TEST_PATH:
        for item in contents:
            item_path = os.path.join(COMP_PATH, item)
            if os.path.isdir(item_path):
                files = os.listdir(item_path)
                if files and any(f.endswith(('.jpg', '.png', '.jpeg')) for f in files):
                    TEST_PATH = item_path
                    print(f"✅ Using as test folder: {TEST_PATH}")
                    break
    
    if not TEST_PATH:
        raise Exception("❌ Could not find test images folder!")
    
    # Create output directory
    OUTPUT_PATH = '/kaggle/working/corrected_images'
    os.makedirs(OUTPUT_PATH, exist_ok=True)
    print(f"✅ Output folder created: {OUTPUT_PATH}")
    
    # Count test images
    test_files = []
    for ext in ['*.jpg', '*.png', '*.jpeg']:
        test_files.extend(Path(TEST_PATH).glob(ext))
    
    print(f"📸 Total test images found: {len(test_files)}")
    
    return COMP_PATH, TRAIN_PATH, TEST_PATH, OUTPUT_PATH, test_files

# Execute path setup
COMP_PATH, TRAIN_PATH, TEST_PATH, OUTPUT_PATH, TEST_FILES = setup_paths()
print("="*60)

🔍 CONFIGURING PATHS AND ENVIRONMENT
✅ Competition path: /kaggle/input/automatic-lens-correction
✅ Train folder found: /kaggle/input/automatic-lens-correction/lens-correction-train-cleaned
✅ Test folder found: /kaggle/input/automatic-lens-correction/test-originals
✅ Output folder created: /kaggle/working/corrected_images
📸 Total test images found: 1000


# Cell 4: Advanced Lens Correction Engine (AGGRESSIVE MODE)


In [4]:
# =============================================================================
# 🔧 ADVANCED LENS CORRECTION ENGINE - BALANCED MODE v3.1
# =============================================================================

class AdvancedLensCorrector:
    """
    Professional lens correction system with BALANCED parameters
    Optimized for competition metrics
    """
    
    def __init__(self, config=None):
        """
        Initialize with BALANCED configuration
        """
        # BALANCED configuration - NOT TOO AGGRESSIVE
        self.config = {
            # Edge detection parameters
            'canny_threshold1': 40,
            'canny_threshold2': 100,
            'hough_threshold': 70,
            'min_line_length': 70,
            'max_line_gap': 12,
            
            # Enhancement parameters
            'clahe_clip_limit': 2.5,
            'clahe_grid_size': (8, 8),
            'bilateral_diameter': 9,
            'bilateral_sigma_color': 75,
            'bilateral_sigma_space': 75,
            
            # BALANCED distortion parameters
            'distortion_k1_factor': 0.15,
            'distortion_k2_factor': 0.02,
            'distortion_k3_factor': 0.002,
            'line_correction_threshold': 0.4,
            'line_correction_boost': 1.5,
            
            # Output quality
            'output_quality': 100,
        }
        
        if config:
            self.config.update(config)
            
        self.stats = {'processed': 0, 'failed': 0}
        print("✅ AdvancedLensCorrector - BALANCED MODE initialized")
    
    def detect_distortion_parameters(self, image):
        """
        Analyze image and estimate optimal distortion parameters
        """
        # Convert to grayscale
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Multi-scale edge detection
        edges_fine = cv2.Canny(gray, 20, 60)
        edges_medium = cv2.Canny(gray, 40, 100)
        edges_coarse = cv2.Canny(gray, 60, 140)
        
        # Combine edges
        edges = cv2.bitwise_or(edges_fine, edges_medium)
        edges = cv2.bitwise_or(edges, edges_coarse)
        
        # Detect lines
        lines = cv2.HoughLinesP(
            edges, 
            rho=1, 
            theta=np.pi/180, 
            threshold=self.config['hough_threshold'],
            minLineLength=self.config['min_line_length'],
            maxLineGap=self.config['max_line_gap']
        )
        
        if lines is not None and len(lines) > 5:
            return self._estimate_from_lines(lines, gray.shape)
        else:
            return self._estimate_from_gradient(gray)
    
    def _estimate_from_lines(self, lines, image_shape):
        """
        Estimate distortion parameters from detected lines
        """
        angles = []
        lengths = []
        
        for line in lines:
            x1, y1, x2, y2 = line[0]
            angle = np.arctan2(y2 - y1, x2 - x1) * 180 / np.pi
            length = np.sqrt((x2 - x1)**2 + (y2 - y1)**2)
            angles.append(angle)
            lengths.append(length)
        
        # Weight by length
        weights = np.array(lengths) / (np.sum(lengths) + 1e-6)
        
        # Calculate distortion factor
        h, w = image_shape
        angle_std = np.std(angles)
        distortion_factor = angle_std / 45.0
        
        # Scale by image size
        scale = np.sqrt(h * w) / 1000
        
        # Calculate parameters
        k1 = self.config['distortion_k1_factor'] * distortion_factor * scale
        k2 = self.config['distortion_k2_factor'] * distortion_factor * scale
        k3 = self.config['distortion_k3_factor'] * distortion_factor * scale
        
        return {'k1': k1, 'k2': k2, 'k3': k3}
    
    def _estimate_from_gradient(self, gray):
        """
        Estimate distortion parameters from gradient analysis
        """
        # Calculate gradients
        grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=5)
        grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=5)
        
        # Calculate gradient magnitude and direction
        magnitude = np.sqrt(grad_x**2 + grad_y**2)
        angle = np.arctan2(grad_y, grad_x)
        
        h, w = gray.shape
        center = np.array([h/2, w/2])
        
        y, x = np.indices((h, w))
        r = np.sqrt((x - center[1])**2 + (y - center[0])**2)
        r_max = np.sqrt(center[0]**2 + center[1]**2)
        r_norm = r / r_max
        
        # Estimate from angle variation
        angle_variation = np.std(angle[r_norm > 0.4])
        k1 = angle_variation * 1.5 / np.pi
        k2 = k1 / 10
        k3 = k1 / 100
        
        return {'k1': k1, 'k2': k2, 'k3': k3}
    
    def apply_aggressive_correction(self, image, params):
        """
        Apply geometric distortion correction
        """
        h, w = image.shape[:2]
        
        # Camera matrix
        camera_matrix = np.array([
            [w, 0, w/2],
            [0, h, h/2],
            [0, 0, 1]
        ], dtype=np.float32)
        
        # Distortion coefficients
        dist_coeffs = np.array([
            params['k1'], 
            params['k2'], 
            0, 0, 
            params['k3']
        ], dtype=np.float32)
        
        # Get optimal new camera matrix
        new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(
            camera_matrix, 
            dist_coeffs, 
            (w, h), 
            alpha=1, 
            newImgSize=(w, h)
        )
        
        # Apply correction
        corrected = cv2.undistort(
            image, 
            camera_matrix, 
            dist_coeffs, 
            None, 
            new_camera_matrix
        )
        
        # Crop and resize if needed
        x, y, w_roi, h_roi = roi
        if w_roi > 0 and h_roi > 0:
            corrected = corrected[y:y+h_roi, x:x+w_roi]
            corrected = cv2.resize(corrected, (w, h))
        
        return corrected
    
    def enhance_line_straightness_aggressive(self, image):
        """
        Line straightening correction
        """
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image.copy()
        
        # Detect edges
        edges = cv2.Canny(gray, 20, 60, apertureSize=3)
        
        # Detect lines
        lines = cv2.HoughLinesP(
            edges, 
            rho=1, 
            theta=np.pi/180, 
            threshold=40,
            minLineLength=40, 
            maxLineGap=10
        )
        
        if lines is None or len(lines) < 3:
            return image
        
        # Apply additional correction if needed
        params = self.detect_distortion_parameters(gray)
        params['k1'] *= self.config['line_correction_boost']
        params['k2'] *= self.config['line_correction_boost']
        
        return self.apply_aggressive_correction(image, params)
    
    def enhance_image_quality_aggressive(self, image):
        """
        Quality enhancement
        """
        result = image.copy()
        
        if len(image.shape) == 3:
            # Convert to LAB
            lab = cv2.cvtColor(result, cv2.COLOR_BGR2LAB)
            l, a, b = cv2.split(lab)
            
            # Apply CLAHE
            clahe = cv2.createCLAHE(
                clipLimit=self.config['clahe_clip_limit'],
                tileGridSize=self.config['clahe_grid_size']
            )
            l_enhanced = clahe.apply(l)
            
            # Merge
            lab_enhanced = cv2.merge([l_enhanced, a, b])
            result = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
            
            # Bilateral filter
            result = cv2.bilateralFilter(
                result,
                d=self.config['bilateral_diameter'],
                sigmaColor=self.config['bilateral_sigma_color'],
                sigmaSpace=self.config['bilateral_sigma_space']
            )
            
            # Sharpening
            kernel = np.array([
                [-1, -1, -1],
                [-1, 9, -1],
                [-1, -1, -1]
            ])
            result = cv2.filter2D(result, -1, kernel)
        
        return result
    
    # ===== MAIN PROCESSING FUNCTION =====
    def process_single_image_balanced(self, image_path):
        """
        Complete BALANCED processing pipeline
        """
        try:
            # Read image
            image = cv2.imread(str(image_path))
            if image is None:
                self.stats['failed'] += 1
                return None
            
            # Step 1: Detect distortion
            params = self.detect_distortion_parameters(image)
            
            # Step 2: Apply correction
            corrected = self.apply_aggressive_correction(image, params)
            
            # Step 3: Line straightening
            line_corrected = self.enhance_line_straightness_aggressive(corrected)
            
            # Step 4: Quality enhancement
            enhanced = self.enhance_image_quality_aggressive(line_corrected)
            
            self.stats['processed'] += 1
            return enhanced
            
        except Exception as e:
            print(f"❌ Error processing image: {e}")
            self.stats['failed'] += 1
            return None

# Cell 5: Submission File Generator


In [5]:
# =============================================================================
# 📦 SUBMISSION FILE GENERATOR
# =============================================================================

class SubmissionGenerator:
    """
    Generate competition submission files with multiple options
    """
    
    def __init__(self, output_path):
        self.output_path = Path(output_path)
        self.submission_file = '/kaggle/working/submission.csv'
        self.zip_file = '/kaggle/working/corrected_images.zip'
        self.final_zip = '/kaggle/working/corrected_images_final.zip'
        
    def create_submission_csv(self):
        """
        Create submission.csv file
        """
        corrected_images = list(self.output_path.glob('*.*'))
        
        if not corrected_images:
            print("❌ No corrected images found!")
            return False
        
        submission_data = []
        for img_path in corrected_images:
            image_id = img_path.stem
            submission_data.append([image_id, 0.0])
        
        submission_df = pd.DataFrame(
            submission_data, 
            columns=['image_id', 'score']
        )
        submission_df.to_csv(self.submission_file, index=False)
        
        print(f"✅ Created submission.csv with {len(submission_data)} images")
        return True
    
    def create_zip_archive(self, quality=95):
        """
        Create zip archive with specified quality
        """
        corrected_images = list(self.output_path.glob('*.*'))
        
        if not corrected_images:
            print("❌ No corrected images found!")
            return False
        
        if quality < 100:
            # Compress with specified quality
            temp_dir = '/kaggle/working/temp_compressed'
            os.makedirs(temp_dir, exist_ok=True)
            
            for img_path in tqdm(corrected_images, desc="Compressing"):
                img = cv2.imread(str(img_path))
                out_path = os.path.join(temp_dir, img_path.name)
                cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, quality])
            
            with zipfile.ZipFile(self.zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for img_path in Path(temp_dir).glob('*.jpg'):
                    zipf.write(img_path, arcname=img_path.name)
        else:
            # No compression
            with zipfile.ZipFile(self.zip_file, 'w', zipfile.ZIP_DEFLATED) as zipf:
                for img_path in tqdm(corrected_images, desc="Zipping"):
                    zipf.write(img_path, arcname=img_path.name)
        
        size_mb = os.path.getsize(self.zip_file) / (1024 * 1024)
        print(f"✅ Created ZIP: {size_mb:.2f} MB (Quality: {quality}%)")
        return True
    
    def create_high_quality_zip(self):
        """
        Create maximum quality zip (100%)
        """
        corrected_images = list(self.output_path.glob('*.*'))
        
        if not corrected_images:
            print("❌ No corrected images found!")
            return False
        
        # Create fixed folder
        fixed_folder = '/kaggle/working/corrected_images_fixed'
        os.makedirs(fixed_folder, exist_ok=True)
        
        # Save with maximum quality
        for img_path in tqdm(corrected_images, desc="Fixing images"):
            img = cv2.imread(str(img_path))
            out_path = os.path.join(fixed_folder, img_path.name)
            cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 100])
        
        # Create zip
        with zipfile.ZipFile(self.final_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for img_path in tqdm(Path(fixed_folder).glob('*.jpg'), desc="Zipping"):
                zipf.write(img_path, arcname=img_path.name)
        
        size_mb = os.path.getsize(self.final_zip) / (1024 * 1024)
        print(f"✅ High Quality ZIP: {size_mb:.2f} MB")
        return True
    
    def get_file_info(self):
        """Return information about generated files"""
        info = {}
        
        if os.path.exists(self.submission_file):
            info['submission'] = {
                'size': os.path.getsize(self.submission_file),
                'rows': len(pd.read_csv(self.submission_file))
            }
        
        if os.path.exists(self.zip_file):
            info['zip'] = {
                'size_mb': os.path.getsize(self.zip_file) / (1024 * 1024),
                'path': self.zip_file
            }
        
        if os.path.exists(self.final_zip):
            info['final_zip'] = {
                'size_mb': os.path.getsize(self.final_zip) / (1024 * 1024),
                'path': self.final_zip
            }
        
        return info

# Cell 6: Main Execution Function (AGGRESSIVE MODE)


In [6]:
# =============================================================================
# 🚀 MAIN EXECUTION FUNCTION - BALANCED MODE v3.1
# =============================================================================

def main_balanced():
    """
    Main execution with BALANCED correction parameters
    """
    start_time = datetime.now()
    
    print("\n" + "="*80)
    print("🚀 STARTING BALANCED LENS CORRECTION PIPELINE v3.1")
    print("="*80)
    
    # Get test files
    test_files = TEST_FILES
    print(f"📸 Found {len(test_files)} test images to process")
    
    if len(test_files) == 0:
        print("❌ No test images found! Exiting...")
        return
    
    # Initialize processor
    print("\n⚙️ Initializing BALANCED correction engine...")
    corrector = AdvancedLensCorrector()
    
    # Process images with progress bar
    print("\n🖼️ Processing images with BALANCED parameters...")
    successful = 0
    
    for i, test_file in enumerate(tqdm(test_files, desc="Progress", unit="img")):
        try:
            # Process single image with balanced pipeline
            corrected = corrector.process_single_image_balanced(test_file)
            
            if corrected is not None:
                # Save corrected image
                output_path = os.path.join(OUTPUT_PATH, test_file.name)
                cv2.imwrite(output_path, corrected, [cv2.IMWRITE_JPEG_QUALITY, 100])
                successful += 1
            
            # Show progress every 100 images
            if (i + 1) % 100 == 0:
                elapsed = (datetime.now() - start_time).total_seconds()
                rate = (i + 1) / elapsed
                print(f"\n📊 Progress: {i + 1}/{len(test_files)} | "
                      f"Rate: {rate:.2f} img/s | "
                      f"Success: {successful}/{i + 1}")
                
        except Exception as e:
            print(f"\n❌ Error on {test_file.name}: {str(e)}")
            continue
    
    # Final statistics
    elapsed_time = (datetime.now() - start_time).total_seconds()
    
    print("\n" + "="*80)
    print("✅ BALANCED PROCESSING COMPLETE")
    print("="*80)
    print(f"""
    ⏱️  Total time:     {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)
    📸 Total images:    {len(test_files)}
    ✅ Successful:       {successful}
    ❌ Failed:           {len(test_files) - successful}
    ⚡ Average rate:     {len(test_files)/elapsed_time:.2f} img/s
    """)
    
    # Generate submission files
    if successful > 0:
        print("\n📦 Generating submission files...")
        generator = SubmissionGenerator(OUTPUT_PATH)
        
        # Create submission CSV
        generator.create_submission_csv()
        
        # Create standard ZIP (95% quality)
        print("\n📦 Creating Standard ZIP (95% quality)...")
        generator.create_zip_archive(quality=95)
        
        # Create high quality ZIP (100% quality)
        print("\n📦 Creating High Quality ZIP (100% quality)...")
        generator.create_high_quality_zip()
        
        # Show file info
        info = generator.get_file_info()
        
        print("\n" + "="*80)
        print("🎯 SUBMISSION FILES READY")
        print("="*80)
        
        if 'zip' in info:
            print(f"📦 Standard ZIP (95%): {info['zip']['size_mb']:.2f} MB")
        if 'final_zip' in info:
            print(f"📦 High Quality ZIP (100%): {info['final_zip']['size_mb']:.2f} MB")
        if 'submission' in info:
            print(f"📄 submission.csv: {info['submission']['rows']} entries")
        
        print("\n" + "⭐"*40)
        print("NEXT STEPS:")
        print("⭐"*40)
        print("""
    1️⃣  Try High Quality ZIP first (100% quality)
    2️⃣  If too large (>500MB), use Standard ZIP (95% quality)
    3️⃣  Upload to: https://bounty.autohdr.com
    4️⃣  Download submission.csv and upload to Kaggle
    
    ⏰ Deadline: Today before midnight
    🏆 Target Score: >0.80
        """)

# Execute main function
main_balanced()


🚀 STARTING BALANCED LENS CORRECTION PIPELINE v3.1
📸 Found 1000 test images to process

⚙️ Initializing BALANCED correction engine...
✅ AdvancedLensCorrector - BALANCED MODE initialized

🖼️ Processing images with BALANCED parameters...


Progress:  10%|█         | 100/1000 [01:21<12:27,  1.20img/s]


📊 Progress: 100/1000 | Rate: 1.23 img/s | Success: 100/100


Progress:  20%|██        | 200/1000 [02:42<11:26,  1.16img/s]


📊 Progress: 200/1000 | Rate: 1.23 img/s | Success: 200/200


...


📊 Progress: 1000/1000 | Rate: 1.26 img/s | Success: 1000/1000

✅ BALANCED PROCESSING COMPLETE

    ⏱️  Total time:     791.91 seconds (13.20 minutes)
    📸 Total images:    1000
    ✅ Successful:       1000
    ❌ Failed:           0
    ⚡ Average rate:     1.26 img/s
    

📦 Generating submission files...
✅ Created submission.csv with 1000 images

📦 Creating Standard ZIP (95% quality)...


Compressing: 100%|██████████| 1000/1000 [00:49<00:00, 20.19it/s]


✅ Created ZIP: 957.01 MB (Quality: 95%)

📦 Creating High Quality ZIP (100% quality)...


Fixing images: 100%|██████████| 1000/1000 [00:57<00:00, 17.40it/s]
Zipping: 1000it [01:22, 12.12it/s]

✅ High Quality ZIP: 2046.92 MB

🎯 SUBMISSION FILES READY
📦 Standard ZIP (95%): 957.01 MB
📦 High Quality ZIP (100%): 2046.92 MB
📄 submission.csv: 1000 entries

⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐
NEXT STEPS:
⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐

    1️⃣  Try High Quality ZIP first (100% quality)
    2️⃣  If too large (>500MB), use Standard ZIP (95% quality)
    3️⃣  Upload to: https://bounty.autohdr.com
    4️⃣  Download submission.csv and upload to Kaggle
    
    ⏰ Deadline: Today before midnight
    🏆 Target Score: >0.80
        


# Cell 7: Performance Optimization & System Info


In [7]:
# =============================================================================
# ⚡ PERFORMANCE OPTIMIZATION & SYSTEM INFORMATION
# =============================================================================

print("="*60)
print("⚡ SYSTEM PERFORMANCE ANALYSIS")
print("="*60)

# CPU Information
cpu_count = multiprocessing.cpu_count()
print(f"✅ CPU Cores: {cpu_count}")

# Memory Information
memory = psutil.virtual_memory()
print(f"✅ RAM Total: {memory.total / (1024**3):.2f} GB")
print(f"✅ RAM Available: {memory.available / (1024**3):.2f} GB")

# GPU Information
try:
    result = subprocess.run(
        ['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], 
        capture_output=True, text=True
    )
    if result.returncode == 0:
        print(f"✅ GPU: {result.stdout.strip()}")
except:
    print("ℹ️ GPU information not available")

print("\n💡 OPTIMIZATION TIPS:")
print("• BALANCED mode enabled - optimized for competition")
print("• Processing 1000 images in ~12-15 minutes")
print("• Two ZIP files created: Standard (95%) and High Quality (100%)")
print("• Monitor memory usage during processing")

⚡ SYSTEM PERFORMANCE ANALYSIS
✅ CPU Cores: 4
✅ RAM Total: 31.35 GB
✅ RAM Available: 30.02 GB
✅ GPU: Tesla P100-PCIE-16GB, 16384 MiB

💡 OPTIMIZATION TIPS:
• BALANCED mode enabled - optimized for competition
• Processing 1000 images in ~12-15 minutes
• Two ZIP files created: Standard (95%) and High Quality (100%)
• Monitor memory usage during processing


# Cell 8: Verification & Diagnostics


In [8]:
# =============================================================================
# ✅ VERIFICATION & DIAGNOSTICS
# =============================================================================

print("="*60)
print("✅ VERIFYING OUTPUTS")
print("="*60)

def verify_all_outputs():
    """Comprehensive verification of all outputs"""
    
    # Check working directory
    working_dir = '/kaggle/working'
    print(f"\n📁 Working directory contents:")
    for item in sorted(os.listdir(working_dir)):
        item_path = os.path.join(working_dir, item)
        if os.path.isdir(item_path):
            files = list(Path(item_path).glob('*.*'))
            print(f"  📂 {item}/ ({len(files)} files)")
        else:
            size = os.path.getsize(item_path) / (1024 * 1024) if item.endswith('.zip') else 0
            if item.endswith('.zip'):
                print(f"  📦 {item} ({size:.2f} MB)")
            elif item.endswith('.csv'):
                try:
                    df = pd.read_csv(item_path)
                    print(f"  📄 {item} ({len(df)} entries)")
                except:
                    print(f"  📄 {item}")
            else:
                print(f"  📄 {item}")
    
    # Check corrected_images folder
    img_folder = Path('/kaggle/working/corrected_images')
    if img_folder.exists():
        images = list(img_folder.glob('*.jpg'))
        print(f"\n✅ corrected_images: {len(images)} images")
        if images:
            print(f"📝 Sample: {[f.name for f in images[:3]]}")
    
    # Check submission files
    sub_file = '/kaggle/working/submission.csv'
    if os.path.exists(sub_file):
        try:
            df = pd.read_csv(sub_file)
            print(f"\n✅ submission.csv: {len(df)} entries")
            print(f"📝 Columns: {list(df.columns)}")
        except:
            print(f"\n✅ submission.csv exists")
    
    # Check ZIP files
    zip_files = list(Path(working_dir).glob('*.zip'))
    if zip_files:
        print(f"\n📦 ZIP files found:")
        for zf in zip_files:
            size = os.path.getsize(zf) / (1024 * 1024)
            print(f"   • {zf.name}: {size:.2f} MB")

# Run verification
verify_all_outputs()

print("\n" + "="*60)
print("🎯 READY FOR SUBMISSION!")
print("="*60)

✅ VERIFYING OUTPUTS

📁 Working directory contents:
  📄 __notebook__.ipynb
  📂 corrected_images/ (1000 files)
  📦 corrected_images.zip (957.01 MB)
  📦 corrected_images_final.zip (2046.92 MB)
  📂 corrected_images_fixed/ (1000 files)
  📄 submission.csv (1000 entries)
  📂 temp_compressed/ (1000 files)

✅ corrected_images: 1000 images
📝 Sample: ['fe6a7f74-e126-4bb6-ae94-76e2c3792c6c_g18.jpg', '10a0779a-b9ca-4fbb-9cdf-cab54ea85eeb_g2.jpg', '10a0779a-b9ca-4fbb-9cdf-cab54ea85eeb_g10.jpg']

✅ submission.csv: 1000 entries
📝 Columns: ['image_id', 'score']

📦 ZIP files found:
   • corrected_images.zip: 957.01 MB
   • corrected_images_final.zip: 2046.92 MB

🎯 READY FOR SUBMISSION!


# Cell 9: High Quality ZIP Creator (100% Quality)


In [9]:
# =============================================================================
# 🔧 HIGH QUALITY ZIP CREATOR - 100% QUALITY
# =============================================================================

import cv2
import os
import zipfile
from pathlib import Path
from tqdm import tqdm

print("="*60)
print("🔧 CREATING HIGH QUALITY ZIP (100%)")
print("="*60)

# Paths
input_folder = '/kaggle/working/corrected_images'
output_folder = '/kaggle/working/corrected_images_fixed'
output_zip = '/kaggle/working/corrected_images_final.zip'

if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
else:
    os.makedirs(output_folder, exist_ok=True)

    # Get images
    image_files = list(Path(input_folder).glob('*.jpg'))
    print(f"📸 Total images: {len(image_files)}")

    # Save with MAXIMUM quality
    successful = 0
    for img_path in tqdm(image_files, desc="Processing images"):
        try:
            img = cv2.imread(str(img_path))
            if img is not None:
                output_path = os.path.join(output_folder, img_path.name)
                cv2.imwrite(output_path, img, [cv2.IMWRITE_JPEG_QUALITY, 100])
                successful += 1
        except Exception as e:
            print(f"Error: {e}")
            continue

    print(f"\n✅ Saved: {successful} images at 100% quality")

    # Create ZIP
    if successful > 0:
        print("\n📦 Creating ZIP file...")
        with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for img_path in tqdm(list(Path(output_folder).glob('*.jpg')), desc="Zipping"):
                zipf.write(img_path, arcname=img_path.name)

        # Show result
        size_mb = os.path.getsize(output_zip) / (1024 * 1024)
        print(f"\n✅ High Quality ZIP created: {output_zip}")
        print(f"📦 Size: {size_mb:.2f} MB")

        if size_mb > 500:
            print("⚠️ File >500MB - Use Standard ZIP instead")
        else:
            print("✅ File ready for upload!")

🔧 CREATING HIGH QUALITY ZIP (100%)
📸 Total images: 1000


Processing images: 100%|██████████| 1000/1000 [00:58<00:00, 17.08it/s]



✅ Saved: 1000 images at 100% quality

📦 Creating ZIP file...


Zipping: 100%|██████████| 1000/1000 [01:21<00:00, 12.23it/s]



✅ High Quality ZIP created: /kaggle/working/corrected_images_final.zip
📦 Size: 2046.92 MB
⚠️ File >500MB - Use Standard ZIP instead


# Cell 10: Standard ZIP Creator (95% Quality) - Under 500MB


In [10]:
# =============================================================================
# 📦 STANDARD ZIP CREATOR - 95% QUALITY (<500MB)
# =============================================================================

import cv2
import os
import zipfile
from pathlib import Path
from tqdm import tqdm

print("="*60)
print("📦 CREATING STANDARD ZIP (95%)")
print("="*60)

# Paths
input_folder = '/kaggle/working/corrected_images'
temp_folder = '/kaggle/working/temp_compressed'
output_zip = '/kaggle/working/corrected_images_ready.zip'

if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
else:
    os.makedirs(temp_folder, exist_ok=True)

    # Get images
    image_files = list(Path(input_folder).glob('*.jpg'))
    print(f"📸 Total images: {len(image_files)}")

    # Compress with 95% quality
    for img_path in tqdm(image_files, desc="Compressing"):
        img = cv2.imread(str(img_path))
        out_path = os.path.join(temp_folder, img_path.name)
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 95])

    # Create ZIP
    print("\n📦 Creating ZIP file...")
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for img_path in tqdm(Path(temp_folder).glob('*.jpg'), desc="Zipping"):
            zipf.write(img_path, arcname=img_path.name)

    # Show result
    size_mb = os.path.getsize(output_zip) / (1024 * 1024)
    print(f"\n✅ Standard ZIP created: {output_zip}")
    print(f"📦 Size: {size_mb:.2f} MB")

    if size_mb <= 500:
        print("✅ Ready for upload!")
    else:
        print("⚠️ Still >500MB - Try 85% quality")

📦 CREATING STANDARD ZIP (95%)
📸 Total images: 1000


Compressing: 100%|██████████| 1000/1000 [00:49<00:00, 20.24it/s]



📦 Creating ZIP file...


Zipping: 1000it [00:41, 24.07it/s]


✅ Standard ZIP created: /kaggle/working/corrected_images_ready.zip
📦 Size: 957.01 MB
⚠️ Still >500MB - Try 85% quality


# Cell 11: Emergency Compressor (85% Quality) - Guaranteed <500MB


In [11]:
# =============================================================================
# 🗜️ EMERGENCY COMPRESSOR - 85% QUALITY (GUARANTEED <500MB)
# =============================================================================

import cv2
import os
import zipfile
from pathlib import Path
from tqdm import tqdm

print("="*60)
print("🗜️ EMERGENCY COMPRESSION - 85% QUALITY")
print("="*60)

# Paths
input_folder = '/kaggle/working/corrected_images'
temp_folder = '/kaggle/working/temp_emergency'
output_zip = '/kaggle/working/corrected_images_small.zip'

if not os.path.exists(input_folder):
    print(f"❌ Input folder not found: {input_folder}")
else:
    os.makedirs(temp_folder, exist_ok=True)

    # Get images
    image_files = list(Path(input_folder).glob('*.jpg'))
    print(f"📸 Total images: {len(image_files)}")

    # Compress with 85% quality
    for img_path in tqdm(image_files, desc="Compressing"):
        img = cv2.imread(str(img_path))
        out_path = os.path.join(temp_folder, img_path.name)
        cv2.imwrite(out_path, img, [cv2.IMWRITE_JPEG_QUALITY, 85])

    # Create ZIP
    with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for img_path in Path(temp_folder).glob('*.jpg'):
            zipf.write(img_path, arcname=img_path.name)

    # Show result
    size_mb = os.path.getsize(output_zip) / (1024 * 1024)
    print(f"\n✅ Emergency ZIP: {size_mb:.2f} MB")
    print("✅ Guaranteed <500MB - Ready for upload!")

🗜️ EMERGENCY COMPRESSION - 85% QUALITY
📸 Total images: 1000


Compressing: 100%|██████████| 1000/1000 [00:45<00:00, 21.85it/s]



✅ Emergency ZIP: 455.38 MB
✅ Guaranteed <500MB - Ready for upload!


In [12]:
# =============================================================================
# 🧹 CLEANUP - DELETE OLD FILES
# =============================================================================

import shutil
import os

print("="*60)
print("🧹 CLEANING OLD FILES")
print("="*60)

folders_to_delete = [
    '/kaggle/working/corrected_images',
    '/kaggle/working/corrected_images_fixed',
    '/kaggle/working/temp',
    '/kaggle/working/temp_compressed',
    '/kaggle/working/temp_emergency'
]

for folder in folders_to_delete:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"✅ Deleted: {folder}")

files_to_delete = [
    '/kaggle/working/corrected_images.zip',
    '/kaggle/working/corrected_images_final.zip',
    '/kaggle/working/corrected_images_ready.zip',
    '/kaggle/working/corrected_images_small.zip'
]

for file in files_to_delete:
    if os.path.exists(file):
        os.remove(file)
        print(f"✅ Deleted: {file}")

print("\n✅ Cleanup complete! Ready for fresh processing.")

🧹 CLEANING OLD FILES
✅ Deleted: /kaggle/working/corrected_images
✅ Deleted: /kaggle/working/corrected_images_fixed
✅ Deleted: /kaggle/working/temp_compressed
✅ Deleted: /kaggle/working/temp_emergency
✅ Deleted: /kaggle/working/corrected_images.zip
✅ Deleted: /kaggle/working/corrected_images_final.zip
✅ Deleted: /kaggle/working/corrected_images_ready.zip
✅ Deleted: /kaggle/working/corrected_images_small.zip

✅ Cleanup complete! Ready for fresh processing.
